# Importing libraries
### Done by Gabriel, Colby, Heejun

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import gc

# import preprocessing classes
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

# feature selection
from sklearn.feature_selection import chi2, mutual_info_regression
from sklearn.feature_selection import SelectKBest, chi2

# import models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

# import train test split
from sklearn.model_selection import train_test_split

# feature importance
from sklearn.inspection import permutation_importance

# import metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error

# functions to save data
from joblib import dump, load

# read data
## mount google drive
from google.colab import drive
drive.mount("/content/drive/")
%cd '/content/drive/MyDrive/DS440/dataset/Data/'
!pwd

## read in files
dir = '/content/drive/MyDrive/DS440/dataset/Data/'
cnx = sqlite3.connect(dir + 'wildfire.sqlite')
cursor = cnx.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='Fires';")
tables = cursor.fetchall()
Fires = pd.read_sql_query("SELECT * FROM 'Fires'", cnx).set_index('FOD_ID')
gc.collect()
# Drop unnecessary column
Fires.drop(columns=['FPA_ID','SOURCE_SYSTEM_TYPE','SOURCE_SYSTEM','NWCG_REPORTING_AGENCY','NWCG_REPORTING_UNIT_ID',
                    'NWCG_REPORTING_UNIT_NAME','SOURCE_REPORTING_UNIT','SOURCE_REPORTING_UNIT_NAME','LOCAL_FIRE_REPORT_ID',
                    'LOCAL_INCIDENT_ID','FIRE_CODE','FIRE_NAME','ICS_209_PLUS_INCIDENT_JOIN_ID','ICS_209_PLUS_COMPLEX_JOIN_ID',
                    'MTBS_ID','MTBS_FIRE_NAME','COMPLEX_NAME','OWNER_DESCR','NWCG_CAUSE_CLASSIFICATION',
                    'NWCG_CAUSE_AGE_CATEGORY','FIRE_SIZE_CLASS','CONT_TIME','CONT_DATE'],inplace=True)

Mounted at /content/drive/
/content/drive/MyDrive/DS440/dataset/Data
/content/drive/MyDrive/DS440/dataset/Data


In [2]:
Fires.head()

,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,NWCG_GENERAL_CAUSE,CONT_DOY,FIRE_SIZE,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_CODE,FIPS_NAME
FOD_ID,,,,,,,,,,,,,
1,2005,2/2/2005 0:00,33,1300.0,Power generation/transmission/distribution,33.0,0.10,40.036944,-121.005833,CA,63,06063,Plumas County
2,2004,5/12/2004 0:00,133,845.0,Natural,133.0,0.25,38.933056,-120.404444,CA,61,06061,Placer County
3,2004,5/31/2004 0:00,152,1921.0,Debris and open burning,152.0,0.10,38.984167,-120.735556,CA,17,06017,El Dorado County
4,2004,6/28/2004 0:00,180,1600.0,Natural,185.0,0.10,38.559167,-119.913333,CA,3,06003,Alpine County
5,2004,6/28/2004 0:00,180,1600.0,Natural,185.0,0.10,38.559167,-119.933056,CA,3,06003,Alpine County


# Data Preprocessing
## convert all features into usable format
## Use random sample from originial dataset due to high computational cost
## Take 1% of original data
## Purpose : 
1. To handle big data
2. To debug and to see that the code actually works

In [3]:
# Later just remove this line of code when using original dataset - Heejun Code
Fires = Fires.sample(frac=0.01,replace=False)

In [4]:
Fires

,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,NWCG_GENERAL_CAUSE,CONT_DOY,FIRE_SIZE,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_CODE,FIPS_NAME
FOD_ID,,,,,,,,,,,,,
400364436,2018,4/9/2018 0:00,99,1112.0,Missing data/not specified/undetermined,99.0,0.10,43.832925,-91.248993,WI,La Crosse,55063,La Crosse County
313390,2005,7/21/2005 0:00,202,1936.0,Natural,204.0,0.10,36.409700,-113.409700,AZ,Mohave,04015,Mohave County
1142197,1994,3/20/1994 0:00,79,NaN,Debris and open burning,NaN,0.20,35.521700,-81.365000,NC,None,None,None
400300530,2018,5/4/2018 0:00,124,2042.0,Recreation and ceremony,124.0,0.01,34.318333,-110.771667,AZ,005,04005,Coconino County
193884,1998,6/14/1998 0:00,165,2315.0,Arson/incendiarism,165.0,0.10,41.066500,-123.684500,CA,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
356932,2001,8/7/2001 0:00,219,1500.0,Natural,219.0,0.10,40.224400,-112.146600,UT,Tooele,49045,Tooele County
459459,2005,2/18/2005 0:00,49,NaN,Debris and open burning,NaN,0.13,31.786882,-81.495670,GA,Liberty,13179,Liberty County
23943,2007,7/10/2007 0:00,191,1016.0,Natural,194.0,0.50,48.621389,-119.945833,WA,47,53047,Okanogan County


# Colby's Coding

In [5]:
# extract month from discovery date
def get_first_element(date_list):
  return date_list[0]

Fires['DISCOVERY_DATE'] = Fires['DISCOVERY_DATE'].astype("string")
Fires['DISCOVERY_LIST'] = Fires['DISCOVERY_DATE'].str.split(pat='/')
Fires['DISCOVERY_MONTH'] = Fires['DISCOVERY_LIST'].map(get_first_element)
Fires['DISCOVERY_MONTH'] = Fires['DISCOVERY_MONTH'].astype("int64")

# encode causes into integers
Fires['NWCG_GENERAL_CAUSE'] = Fires['NWCG_GENERAL_CAUSE'].astype("string")
cause_enc = LabelEncoder()
Fires['NWCG_CAUSE_CLASSIFICATION_ORD'] = cause_enc.fit_transform(Fires['NWCG_GENERAL_CAUSE'])


# figure out number of days to contain the fire
Fires['DAYS_TO_CONT'] = Fires['CONT_DOY'] - Fires['DISCOVERY_DOY']
##### if it was contained the next year, need to do something about that
def convert_negative_days(day):
  if day < 0:
    return 365 + day
  else:
    return day

Fires['DAYS_TO_CONT'] = Fires['DAYS_TO_CONT'].map(convert_negative_days)


# encode state categories into integers
Fires['STATE'] = Fires['STATE'].astype("string")
state_enc = LabelEncoder()
Fires['STATE_ORD'] = state_enc.fit_transform(Fires['STATE'])

# what to do about counties? Use fips code (must be an int)
Fires['FIPS_CODE'] = Fires['FIPS_CODE'].astype("string")
Fires['FIPS_CODE'].fillna('1000000',inplace=True)
Fires['FIPS_CODE'] = Fires['FIPS_CODE'].astype("int64")

# fill NA with mean of day to containment (mean = 0.92)
Fires['DAYS_TO_CONT'].fillna(value=Fires['DAYS_TO_CONT'].mean(),inplace=True)


# drop date and list
Fires.drop(columns=['DISCOVERY_DATE','DISCOVERY_LIST','NWCG_GENERAL_CAUSE',
                    'CONT_DOY','DISCOVERY_DOY','STATE','FIPS_NAME',
                    'COUNTY'],inplace=True)
gc.collect()

30

In [6]:
Fires.head()

,FIRE_YEAR,DISCOVERY_TIME,FIRE_SIZE,LATITUDE,LONGITUDE,FIPS_CODE,DISCOVERY_MONTH,NWCG_CAUSE_CLASSIFICATION_ORD,DAYS_TO_CONT,STATE_ORD
FOD_ID,,,,,,,,,,
400364436,2018,1112.0,0.10,43.832925,-91.248993,55063,4,5,0.000000,49
313390,2005,1936.0,0.10,36.409700,-113.409700,4015,7,7,2.000000,3
1142197,1994,NaN,0.20,35.521700,-81.365000,1000000,3,1,0.833678,27
400300530,2018,2042.0,0.01,34.318333,-110.771667,4005,5,11,0.000000,3
193884,1998,2315.0,0.10,41.066500,-123.684500,1000000,6,0,0.000000,4


#** Code by Heejun Son**
### Need to handle missing values in DISCOVERY_TIME column 
### Need to compare MICE vs KNN vs Replacement with the most frequent number
I did not drop DISCOVERY_TIME column.


In [18]:
# Find missing values in DISCOVERY_TIME column
# Replace NaN with most_frequent
Fires['DISCOVERY_TIME'] = Fires[['DISCOVERY_TIME']].fillna(Fires[['DISCOVERY_TIME']].mode().iloc[0])
Fires

,FIRE_YEAR,DISCOVERY_TIME,FIRE_SIZE,LATITUDE,LONGITUDE,FIPS_CODE,DISCOVERY_MONTH,NWCG_CAUSE_CLASSIFICATION_ORD,DAYS_TO_CONT,STATE_ORD
FOD_ID,,,,,,,,,,
400364436,2018,1112.0,0.10,43.832925,-91.248993,55063,4,5,0.000000,49
313390,2005,1936.0,0.10,36.409700,-113.409700,4015,7,7,2.000000,3
1142197,1994,1500.0,0.20,35.521700,-81.365000,1000000,3,1,0.833678,27
400300530,2018,2042.0,0.01,34.318333,-110.771667,4005,5,11,0.000000,3
193884,1998,2315.0,0.10,41.066500,-123.684500,1000000,6,0,0.000000,4
...,...,...,...,...,...,...,...,...,...,...
356932,2001,1500.0,0.10,40.224400,-112.146600,49045,8,7,0.000000,45
459459,2005,1500.0,0.13,31.786882,-81.495670,13179,2,1,0.833678,10
23943,2007,1016.0,0.50,48.621389,-119.945833,53047,7,7,3.000000,48


# Create a baseline mse and mae

In [19]:
FIRE_SIZE_PERMUTE = Fires['FIRE_SIZE'].sample(frac=1,random_state=0)

In [20]:
print("MAE: ",mean_absolute_error(FIRE_SIZE_PERMUTE,Fires['FIRE_SIZE']))
print("MSE: ",mean_squared_error(FIRE_SIZE_PERMUTE,Fires['FIRE_SIZE']))

MAE:  168.6056608177958
MSE:  14180690.350972895


# ** Code by Heejun **
## Feature selection information
Later need to convert LONGITUDE to negative values!!

Reason for using abs() : There are no positive values !!
                        
(United States only have negative lonitude values)

In [21]:
# Convert negative value to positive
# Feature Selection must have non-negative values
Fires['LONGITUDE'] = Fires['LONGITUDE'].abs()
Fires

,FIRE_YEAR,DISCOVERY_TIME,FIRE_SIZE,LATITUDE,LONGITUDE,FIPS_CODE,DISCOVERY_MONTH,NWCG_CAUSE_CLASSIFICATION_ORD,DAYS_TO_CONT,STATE_ORD
FOD_ID,,,,,,,,,,
400364436,2018,1112.0,0.10,43.832925,91.248993,55063,4,5,0.000000,49
313390,2005,1936.0,0.10,36.409700,113.409700,4015,7,7,2.000000,3
1142197,1994,1500.0,0.20,35.521700,81.365000,1000000,3,1,0.833678,27
400300530,2018,2042.0,0.01,34.318333,110.771667,4005,5,11,0.000000,3
193884,1998,2315.0,0.10,41.066500,123.684500,1000000,6,0,0.000000,4
...,...,...,...,...,...,...,...,...,...,...
356932,2001,1500.0,0.10,40.224400,112.146600,49045,8,7,0.000000,45
459459,2005,1500.0,0.13,31.786882,81.495670,13179,2,1,0.833678,10
23943,2007,1016.0,0.50,48.621389,119.945833,53047,7,7,3.000000,48


In [22]:
Fires.dtypes

FIRE_YEAR                          int64
DISCOVERY_TIME                   float64
FIRE_SIZE                        float64
LATITUDE                         float64
LONGITUDE                        float64
FIPS_CODE                          int64
DISCOVERY_MONTH                    int64
NWCG_CAUSE_CLASSIFICATION_ORD      int64
DAYS_TO_CONT                     float64
STATE_ORD                          int64
dtype: object

In [23]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import preprocessing
X = Fires[['FIRE_YEAR','LATITUDE','LONGITUDE','FIPS_CODE','DISCOVERY_MONTH','NWCG_CAUSE_CLASSIFICATION_ORD','DAYS_TO_CONT','STATE_ORD','DISCOVERY_TIME']]
y = Fires['FIRE_SIZE']
# Transform FIRE_SIZE
# SelectKBest() cannot handle countinous numbers
Encoder = preprocessing.LabelEncoder()
Firs_size_encoded = Encoder.fit_transform(y)

Best_Features = dict()
for i in range(1,9):
  selector = SelectKBest(chi2, k=i)
  selector.fit(X, Firs_size_encoded)
  Features = list(X.columns[selector.get_support(indices=True)])
  Best_Features[i] = Features
  print("When k= ", i, ",", Features)

When k=  1 , ['FIPS_CODE']
When k=  2 , ['FIPS_CODE', 'DAYS_TO_CONT']
When k=  3 , ['FIPS_CODE', 'DAYS_TO_CONT', 'DISCOVERY_TIME']
When k=  4 , ['FIPS_CODE', 'DAYS_TO_CONT', 'STATE_ORD', 'DISCOVERY_TIME']
When k=  5 , ['LONGITUDE', 'FIPS_CODE', 'DAYS_TO_CONT', 'STATE_ORD', 'DISCOVERY_TIME']
When k=  6 , ['LONGITUDE', 'FIPS_CODE', 'NWCG_CAUSE_CLASSIFICATION_ORD', 'DAYS_TO_CONT', 'STATE_ORD', 'DISCOVERY_TIME']
When k=  7 , ['LATITUDE', 'LONGITUDE', 'FIPS_CODE', 'NWCG_CAUSE_CLASSIFICATION_ORD', 'DAYS_TO_CONT', 'STATE_ORD', 'DISCOVERY_TIME']
When k=  8 , ['LATITUDE', 'LONGITUDE', 'FIPS_CODE', 'DISCOVERY_MONTH', 'NWCG_CAUSE_CLASSIFICATION_ORD', 'DAYS_TO_CONT', 'STATE_ORD', 'DISCOVERY_TIME']


Freatures to predict class:
- fire year, general cause, lat, long

Features to predict cause:
- fire year, size class, lat, long

# MODEL

In [24]:
#### split data into training, validation, and testing
X = Fires[['FIRE_YEAR','LATITUDE','LONGITUDE','FIPS_CODE','DISCOVERY_MONTH','NWCG_CAUSE_CLASSIFICATION_ORD','DAYS_TO_CONT','STATE_ORD','DISCOVERY_TIME']]
y = Fires['FIRE_SIZE']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [25]:
#### list of models to train
# xgboost, linear regression (with and without regularization), random forest, naive bayes, nearest neighbors
# k-means, svm
model_list = [LinearRegression, RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor,
               KNeighborsRegressor, SVR] #GaussianNB, 
model_scores = dict()
for model in model_list:
  # fit model to data
  print(model)
  regressor = model().fit(X_train,y_train)
  y_pred = regressor.predict(X_test)
  model_mse = mean_squared_error(y_pred,y_test)
  model_mae = mean_absolute_error(y_pred,y_test)
  model_scores[model] = (regressor,y_pred,model_mse,model_mae)
  print("MSE = ", model_mse, '\n', "MAE = ", model_mae)

<class 'sklearn.linear_model._base.LinearRegression'>
MSE =  1769588.3514248272 
 MAE =  156.75566703422183
<class 'sklearn.ensemble._forest.RandomForestRegressor'>
MSE =  7697232.9208881585 
 MAE =  173.61298335055324
<class 'sklearn.ensemble._gb.GradientBoostingRegressor'>
MSE =  11148377.020389829 
 MAE =  170.44307937191948
<class 'sklearn.ensemble._weight_boosting.AdaBoostRegressor'>
MSE =  17332798.623477496 
 MAE =  2199.2833400431027
<class 'sklearn.neighbors._regression.KNeighborsRegressor'>
MSE =  3500986.106780888 
 MAE =  117.14794576372864
<class 'sklearn.svm._classes.SVR'>
MSE =  2013142.8392659896 
 MAE =  60.060688564093745


# Save Models

In [ ]:
from joblib import dump, load

In [ ]:
for m in model_scores.keys():
  # save model object
  dump(model_scores[m][0],'/content/drive/MyDrive/DS 440/Models/ModelIs%s.joblib' %str(model_scores[m][0]))
  # save predictions
  dump(model_scores[m][1],'/content/drive/MyDrive/DS 440/Predictions/PredFor%s.joblib' %str(model_scores[m][0]))
  

In [ ]:
# visualize features - might need more notebooks for this




In [ ]:
# develop model using hyperopt



In [ ]:
# use permutation importance on each feature



In [ ]:
 # put results in a dataframe

 

In [ ]:
############### SAVE FOR LATER ###########################
# polynomial features


# determine which ones are useful 
## use mutual information criterion to do this

